# Biomorph
One of the virtual creatures in a computer simulation described by Richard Dawkins in his book [The Blind Watchmaker](https://en.wikipedia.org/wiki/The_Blind_Watchmaker)

*The Blind Watchmaker: Why the Evidence of Evolution Reveals a Universe without Design* is a 1986 book by Richard Dawkins, in which the author presents an explanation of, and argument for, the theory of evolution by means of natural selection.

## Import

In [1]:
!rm -rf mola
!git clone https://github.com/dbt-ethz/mola.git

Cloning into 'mola'...
remote: Enumerating objects: 51, done.
remote: Counting objects: 100% (51/51), done.
remote: Compressing objects: 100% (38/38), done.
remote: Total 1178 (delta 26), reused 34 (delta 13), pack-reused 1127
Receiving objects: 100% (1178/1178), 237.96 KiB | 1.42 MiB/s, done.
Resolving deltas: 100% (750/750), done.


In [0]:
import random 
import math
from IPython.display import HTML, SVG
import mola.renderP5JS as renderer2D
from mola.core import *
from mola import io

## BioMorph Class Definition

In [0]:
class BioMorph:
  '''
  A Biomorph carries his genes as a list of 11 values.
  It can create a mutated copy of its genes and can pass those over to the entire population
  In order to create the phenotype out of the genes, a Biomparh can recursively draw its tree-like geometry
  '''
  def __init__(self):
    self.genes = [0]*11
    self.genes[0] = random.randrange(20) # depth
    self.genes[1] = random.randrange(20) # x-scale levels 3, 6, 9
    self.genes[2] = random.randrange(20) # x-scale levels 1, 4, 7
    self.genes[3] = random.randrange(20) # x-scale levels 2, 5, 8
    self.genes[4] = random.randrange(20) # y-scale levels 3, 6, 9
    self.genes[5] = random.randrange(20) # y-scale levels 1, 4, 7
    self.genes[6] = random.randrange(20) # y-scale levels 2, 5, 8
    self.genes[7] = random.randrange(20) # red
    self.genes[8] = random.randrange(20) # green
    self.genes[9] = random.randrange(20) # blue
    self.genes[10] = random.randrange(20)# width
    
    self.dx =(0,1,1,1,0,-1,-1,-1)# a predefined set of 8 directions, here the x coordinate
    self.dy =(-1,-1,0,1,1,1,0,-1)# a predefined set of 8 directions, here the y coordinate

  def getMutation(self):
    gene = random.randrange(len(self.genes))
    result =list(self.genes)
    # Mutate the gene, either gene+=1 or gene-=1
    if (random.uniform(0,1) > 0.5):
      result[gene] = result[gene] - 1
      if (result[gene] < 0):
        result[gene] = 20
    else:
      result[gene] = result[gene] + 1
      if (result[gene] > 20):
        result[gene] = 0
    return result
  
  def createNewPopulation(self):
    for biomorph in population:
      if biomorph != self:
        biomorph.genes = self.getMutation()

  def draw(self):
    self.maxSegmentLen = 20
    treeDepth = self.genes[0] % 8 + 1# min depth=1, max depth is 10
    initTreeGrowthDirection = 0
    self.lines=[]
    self.drawTree(0, 0, treeDepth, initTreeGrowthDirection)
    return self.lines

  def drawTree(self, x,  y,  treeDepth,  treeGrowthDirection):
    if (treeGrowthDirection < 0):
      treeGrowthDirection += 8
    elif (treeGrowthDirection > 7):
      treeGrowthDirection -= 8
  
    xoffset = self.dx[treeGrowthDirection]
    yoffset = self.dy[treeGrowthDirection]
      
    xGene = (treeDepth % 3) + 1
    yGene = xGene + 3
    i = x + (xoffset * (self.genes[xGene] * 2579) % self.maxSegmentLen)
    j = y + (yoffset * (self.genes[yGene] * 5051) % self.maxSegmentLen)
    r = ((self.genes[7] * 577)%255 + treeDepth * 97) % 255
    g = ((self.genes[8] * 1297)%255 + treeDepth * 97) % 255
    b = ((self.genes[9] * 2089)%255 + treeDepth * 97) % 255
      
    edge=Edge(Vertex(x, y),Vertex(i, j))
    edge.color=[r,g,b]
    edge.strokeWeight=abs(self.genes[10] * treeDepth) % 5 + 1
    self.lines.append(edge)
    # recursion is over
    if treeDepth<0:
      return 0
    # Draw the left and right subtrees recursively.
    self.drawTree(i, j, treeDepth - 1, treeGrowthDirection + 1)
    self.drawTree(i, j, treeDepth - 1, treeGrowthDirection - 1)

##Init

In [0]:
#Initialise the first population of 12 biomorphs
population = []
for i in range(12):
  population.append(BioMorph())

## Grow
Select your favourite biomorph. The computer will create random offsprings and present a new population.
Numbering of offsprings starts with 0, and counts in rows from left to right.

In [14]:
#@title Grow your favourite Biomorph { run: "auto" }
selection = 2 #@param [1,2,3,4,5,6,7,8,9,10,11,12] {type:"raw"}
population[selection-1].createNewPopulation()

renderer2D.beginDraw()
for i in range(len(population)):
  tX=200*(i%4)+100
  tY=150*(i//4)+20
  renderer2D.strokeWeight(1)
  renderer2D.noStroke()
  renderer2D.fill(0,0,0)
  renderer2D.text(str(i+1),tX-3,tY)
  tY+=10
  lines=population[i].draw()
  for line in lines:
    y1=line.v1.y+tY
    y2=line.v2.y+tY
    renderer2D.stroke(line.color[0],line.color[1],line.color[2])
    renderer2D.strokeWeight(line.strokeWeight)
    renderer2D.line(tX+line.v1.x,y1,tX+line.v2.x,y2)
    renderer2D.line(tX-line.v1.x,y1,tX-line.v2.x,y2)
HTML(renderer2D.endDraw())